In [1]:
import sys

import numpy as np
from process_bigraph import pp


sys.path.insert(0, '..')


from biosimulator_processes.services.rest_service import BiosimulationsRestService

/Users/alexanderpatrie/Library/Caches/pypoetry/virtualenvs/biosimulator-processes-JaN5cQMh-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess not available. Error: cannot import name 'fetch_sbml_file' from 'biosimulator_processes.io' (/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/io.py)
GetSbmlStep not available. Error: cannot import name 'fetch_sbml_file' from 'biosimulator_processes.io' (/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/io.py)
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
repressilator_run_id = "61fea4a08c1e3dc95a79802e"

query = "tumor"

result = await BiosimulationsRestService().extract_data(query)

ValueError: invalid literal for int() with base 10: ''

In [3]:
import requests
get_files_url = f'https://api.biosimulations.dev/results/{repressilator_run_id}/download'
headers = {'accept': 'application/json'}
files_resp = requests.get(get_files_url, headers=headers)

In [4]:
files_resp.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
import h5py

def explore_hdf5(file_path):
    with h5py.File(file_path, 'r') as f:
        def print_attrs(name, obj):
            print(f"Name: {name}")
            print("Attributes:")
            for key, val in obj.attrs.items():
                print(f"    {key}: {val}")
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset shape: {obj.shape}")
                print(f"Dataset dtype: {obj.dtype}")
            elif isinstance(obj, h5py.Group):
                print("Group")
            print()
        
        f.visititems(print_attrs)

# Usage example
file_path = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/reports.h5'
explore_hdf5(file_path)

Name: simulation.sedml
Attributes:
    combineArchiveLocation: simulation.sedml
    uri: simulation.sedml
Group

Name: simulation.sedml/Figure_1c
Attributes:
    _type: SedPlot2D
    sedmlDataSetDataTypes: ['float64' 'float64' 'float64' 'float64']
    sedmlDataSetIds: ['data_gen_px' 'data_gen_py' 'data_gen_pz' 'data_gen_time']
    sedmlDataSetLabels: ['data_gen_px' 'data_gen_py' 'data_gen_pz' 'data_gen_time']
    sedmlDataSetNames: ['LacI protein' 'TetR protein' 'cI protein' 'time']
    sedmlDataSetShapes: ['601' '601' '601' '601']
    sedmlId: Figure_1c
    sedmlName: Figure 1c
    uri: simulation.sedml/Figure_1c
Dataset shape: (4, 601)
Dataset dtype: float64

Name: simulation.sedml/report
Attributes:
    _type: SedReport
    sedmlDataSetDataTypes: ['float64' 'float64' 'float64' 'float64' 'float64' 'float64' 'float64']
    sedmlDataSetIds: ['data_set_time' 'data_set_laci_protein' 'data_set_tetr_protein'
 'data_set_ci_protein' 'data_set_laci_mrna' 'data_set_tetr_mrna'
 'data_set_ci_mrn

In [6]:
def read_dataset(file_path, dataset_id):
    with h5py.File(file_path, 'r') as f:
        if dataset_id in f:
            data = f[dataset_id][:]
            print(f"Data from dataset '{dataset_id}':\n{data}")
        else:
            print(f"Dataset '{dataset_id}' not found in the file.")
            
            
read_dataset(file_path, 'data_set_time')

Dataset 'data_set_time' not found in the file.


In [7]:
from biosimulator_processes.services.rest_service import BiosimulationsRestService

In [8]:
from basico import * 


fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/BIOMD0000000012_url.xml'

model = load_model(fp)

In [9]:
get_species(model=model)

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id,transition_time,display_name
name,,,,,,,,,,,,,,,
LacI protein,cell,reactions,#/fl,0.0,0.0,,,0.0,0.0,0.000000,0.000000,Metabolite_0,PX,NaN,LacI protein
TetR protein,cell,reactions,#/fl,0.0,0.0,,,0.0,0.0,138.629436,138.629436,Metabolite_1,PY,0.000000,TetR protein
cI protein,cell,reactions,#/fl,0.0,0.0,,,0.0,0.0,0.000000,0.000000,Metabolite_2,PZ,NaN,cI protein
LacI mRNA,cell,reactions,#/fl,0.0,0.0,,,0.0,0.0,30.000000,30.000000,Metabolite_3,X,0.000000,LacI mRNA
TetR mRNA,cell,reactions,#/fl,20.0,20.0,,,20.0,20.0,23.068528,23.068528,Metabolite_4,Y,0.666667,TetR mRNA
cI mRNA,cell,reactions,#/fl,0.0,0.0,,,0.0,0.0,30.000000,30.000000,Metabolite_5,Z,0.000000,cI mRNA


In [10]:
from biosimulator_processes.processes.copasi_process import CopasiProcess
group_path = 'simulation.sedml/report'
dataset_label = 'cI protein'
outputs = BiosimulationsRestService().read_report_outputs(file_path, dataset_label)




process = CopasiProcess(config={'model': {'model_source': fp}})

Group 'cI protein' not found in the file.
found a filepath


In [11]:
process.inputs()[f'floating_species_concentrations']

{'LacI protein': {'_type': 'float', '_apply': 'set'},
 'TetR protein': {'_type': 'float', '_apply': 'set'},
 'cI protein': {'_type': 'float', '_apply': 'set'},
 'LacI mRNA': {'_type': 'float', '_apply': 'set'},
 'TetR mRNA': {'_type': 'float', '_apply': 'set'},
 'cI mRNA': {'_type': 'float', '_apply': 'set'}}

In [12]:
from biosimulator_processes.verify.core import verify_ode_process_outputs


verification = verify_ode_process_outputs('copasi', file_path, fp)

found a filepath
found a filepath
Species: Time, Num points: 601
Species: LacI protein, Num points: 601
Species: TetR protein, Num points: 601
Species: cI protein, Num points: 601
Species: LacI mRNA, Num points: 601
Species: TetR mRNA, Num points: 601
Species: cI mRNA, Num points: 601


In [ ]:
process.initial_state()['floating_species_concentrations'].keys()

In [ ]:
def is_equal(a, b):
    return a == b or b == a  


from dataclasses import dataclass

from process_bigraph import Process


@dataclass 
class OutputAspectVerification:
    aspect_type: str  # one of: 'names', 'values'. TODO: Add more
    is_verified: bool
    

def create_ode_process_instance(process_name: str, biomodel_id=None, sbml_model_file=None) -> Process:
    module_name = f'{process_name}_process'
    import_statement = f'biosimulator_processes.processes.{module_name}'
    module_paths = module_name.split('_')
    class_name = module_paths[0].replace(module_name[0], module_name[0].upper())
    class_name += module_paths[1].replace(module_paths[1][0], module_paths[1][0].upper())
    module = __import__(
        import_statement, fromlist=[class_name])
    model_source = biomodel_id or sbml_model_file
    bigraph_class = getattr(module, class_name)
    return bigraph_class(config={'model': {'model_source':model_source}})


def verify_ode_process_output_names(process_name: str, source_report_fp: str, biomodel_id: str = None, sbml_model_file: str = None) -> OutputAspectVerification:
    # Get the class from the module
    # TODO: Automatically generate this from the biosimulations rest api
    process = create_ode_process_instance(process_name, biomodel_id, sbml_model_file)
    process_keys = list(process.inputs()['floating_species_concentrations'].keys())
    
    report_outputs = BiosimulationsRestService().read_report_outputs(report_file_path=source_report_fp)
    report_keys = [datum.dataset_label for datum in report_outputs.data]
    for i, val in enumerate(report_keys):
        if report_keys[i].lower() == 'time':
            report_keys.pop(i)
    
    return OutputAspectVerification(aspect_type='names', is_verified=is_equal(report_keys, process_keys))

In [ ]:
from biosimulator_processes.verify.core import verify_ode_process_output_names


verify_ode_process_output_names(process_name='copasi', source_report_fp=file_path, sbml_model_file=fp)

In [ ]:
x = np.array([1, 2, 3])
y = np.array([1, 2, 3.0000001])

In [ ]:
np.allclose(x, y)

In [ ]:
class TokenizedWord:
    def __init__(self, word: str, alphabet=None, language='english'):
        super().__init__()
        self.word = word
        self.alphabet = alphabet or [chr(i) for i in range(ord('a'), ord('z') + 1)]
        self.language = language
        self.data = self.tokenize()
        self.encoded = list(self.data.values())
    
    def __repr__(self):
        return repr(self.tokenize())
    
    def tokenized_word(self):
        return list(self.tokenize().values())
    
    def _create_mapping(self, data):
        return {float(i): letter for i, letter in enumerate(data)}
    
    def transform_data(self, data, r: tuple) -> np.ndarray:
        """Transform the `data` to fit range `r`, where `r=(rangeStart, rangeStop)`"""
        min_orig = min(data)
        max_orig = max(data)
        normalized_data = [a + ((x - min_orig) * (r[1] - r[0]) / (max_orig - min_orig)) for x in data]
        return np.array(normalized_data, dtype='float64')
    
    def normalize(self, data, a, z):
        min_orig = min(data)
        max_orig = max(data)
        
        # Apply min-max normalization
        normalized_data = [a + ((x - min_orig) * (z - a) / (max_orig - min_orig)) for x in data]
        return normalized_data
    
    def tokenize(self):
        # 1. create alphabet mapping (implement this for any given spoken language/music. For example, 12 tones mapped to the harmonic series
        alphabet_data = self.alphabet
        alphabet_mapping = self._create_mapping(alphabet_data)
        
        # 2. transform letter mapping range to fit 0-9, as double digits cannot be verified/read, which in python is 0, 9+1
        alphabet_index = [int(n) for n in list(alphabet_mapping.keys())]
        
        input_range = [int(min(list(alphabet_mapping.keys()))), int(max(list(alphabet_mapping.keys())))]
        start = input_range[0]
        end = input_range[1]
        transformed_letter_map_index = np.linspace(0, 9, end, dtype='float64')
    
        transformed_alphabet_mapping = dict(zip(alphabet_data, transformed_letter_map_index))
        word_letters = [letter for letter in self.word]
            
        return {
            letter: transformed_alphabet_mapping[letter]
            for letter in word_letters}
        

In [ ]:
[letter for letter in 'hello']

In [ ]:
tokenized = TokenizedWord(word='what')

tokenized.tokenized_word()

In [ ]:
tokenize_word('hello')

In [ ]:
def transform_data(data, r: tuple) -> np.ndarray:
    """Transform the `data` to fit range `r`, where `r=(rangeStart, rangeStop)`"""
    min_orig = min(data)
    max_orig = max(data)
    normalized_data = [a + ((x - min_orig) * (r[1] - r[0]) / (max_orig - min_orig)) for x in data]
    return np.array(normalized_data, dtype='float64')


# Original data
data = list(range(26))  # This generates a list [0, 1, 2, ..., 25]

# Define the target range
a = 0
z = 1

# Normalize the data
normalized_data = transform_data(data, (a, z))
print(normalized_data)